## Init

In [27]:
import sys
# print(sys.path)

!python --version

!pip install -q mediapipe

# 모델 파일 다운로드 (Pose Landmarker Lite, Float16, v1)
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

Python 3.12.7


'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


## 실시간으로 MPP 동작

### 실시간

- 왼쪽-오른쪽 구분 로직 추가
- 30초 기준으로 중단

In [ ]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
# VIDEO_PATH = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
MAX_WIDTH = 640  # 영상 최대 너비
TIME_LIMIT = 10  # 시간 제한 (초)
BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준
SIDE_VIS_MARGIN = 0.15          # visibility 합 차이 최소 마진
SIDE_STABILITY_FRAMES = 8       # 연속 프레임 유지 필요 수
MIN_VIS = 0.7 # 최소 visibility
SESSION_MAX_SECONDS = 30 # 30초
session_running = False
session_start_time = None
last_display_frame = None

SIDE_IDX = {
    "left": {
        "HIP": mp_pose.PoseLandmark.LEFT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.LEFT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.LEFT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.LEFT_SHOULDER.value
    },
    "right": {
        "HIP": mp_pose.PoseLandmark.RIGHT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.RIGHT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    }
}

In [ ]:
def joints_visible(landmarks, side, min_vis=MIN_VIS):
    ids = SIDE_IDX[side]
    needed = ["HIP", "KNEE", "ANKLE", "SHOULDER"]
    vis_map = {name: landmarks[ids[name]].visibility for name in needed}
    ok = all(vis_map[n] >= min_vis for n in needed)
    return ok, vis_map

In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [ ]:
def check_side(landmarks, prev_side, stability_counter):
    l_ids = SIDE_IDX["left"]
    r_ids = SIDE_IDX["right"]
    l_sum = (landmarks[l_ids["HIP"]].visibility +
             landmarks[l_ids["KNEE"]].visibility +
             landmarks[l_ids["ANKLE"]].visibility)
    r_sum = (landmarks[r_ids["HIP"]].visibility +
             landmarks[r_ids["KNEE"]].visibility +
             landmarks[r_ids["ANKLE"]].visibility)

    candidate = prev_side
    diff = l_sum - r_sum
    if prev_side is None:
        if l_sum >= r_sum:
            candidate = "left"
        else:
            candidate = "right"
        stability_counter = 0
    else:
        if diff > SIDE_VIS_MARGIN and prev_side != "left":
            candidate = "left"
        elif diff < -SIDE_VIS_MARGIN and prev_side != "right":
            candidate = "right"
        else:
            return prev_side, stability_counter, l_sum, r_sum

        if candidate != prev_side:
            stability_counter += 1
            if stability_counter >= SIDE_STABILITY_FRAMES:
                prev_side = candidate
                stability_counter = 0
            else:
                candidate = prev_side
        else:
            stability_counter = 0

    return candidate, stability_counter, l_sum, r_sum

In [ ]:
def check_knee_angle(landmarks, side):
    ids = SIDE_IDX[side]

    hip = [landmarks[ids["HIP"]].x, landmarks[ids["HIP"]].y]
    knee = [landmarks[ids["KNEE"]].x, landmarks[ids["KNEE"]].y]
    ankle = [landmarks[ids["ANKLE"]].x, landmarks[ids["ANKLE"]].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(hip, knee, ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        # is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        # is_bent_correctly = False

    # return feedback, is_bent_correctly
    return feedback, knee_angle

In [ ]:
def draw_feedback(image, feedback, state, squat_count, extra_lines=None):
    # 스쿼트 상태 및 횟수 표시
    cv2.putText(
        image,
        f"State: {state}, Squat Count: {squat_count}",
        # (50, 50),  # 텍스트 위치 (x, y)
        # cv2.FONT_HERSHEY_SIMPLEX,
        # 1.0, (0, 255, 0), 2, cv2.LINE_AA
        (30, 50),
        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2, cv2.LINE_AA
    )

    # 피드백 정보 표시 (우측 상단)
    cv2.putText(
        image,
        feedback,
        (30, 90),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8, (255,0,0), 2, cv2.LINE_AA
    )

    if extra_lines:
        y = 130
        for line in extra_lines:
            cv2.putText(image, line, (30, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 255), 2, cv2.LINE_AA)
            y += 35

In [ ]:
def resize_image(image, max_width):
    w = image.shape[1]
    if w <= max_width:
        return image
    scale = max_width / w
    h = int(image.shape[0] * scale)
    return cv2.resize(image, (max_width, h))

In [ ]:
def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
    ids = SIDE_IDX[side]
    hip_y = landmarks[ids["HIP"]].y
    knee_y = landmarks[ids["KNEE"]].y
    shoulder_y = landmarks[ids["SHOULDER"]].y

    # 처음 자세의 엉덩이 위치 저장
    if initial_hip_y_dict[side] is None:
        initial_hip_y_dict[side] = hip_y
        print(f"Initial Hip Y: {hip_y:.3f}")
    init_hip = initial_hip_y_dict[side]

    visible_ok, vis_map = joints_visible(landmarks, side)
    if not visible_ok:
        # 스쿼트 판정/카운트 건너뜀
        feedback = "Low visibility - hold still"
        extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]  # 예: HIP:0.63 ...
        return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

    # 허리 펴짐 정도 계산
    back_straightness = abs(hip_y - shoulder_y)

    feedback = ""
    extra = []

    # 상태 전환 로직
    if state == "START":
        feedback = "Check posture..."
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            extra.append("Back: Good")
        else:
            extra.append("Back: Straighten!")
        # 엉덩이 :=: 무릎
        if abs(hip_y - knee_y) < 0.05:
            state = "SIT"
            sit_start_time = time.time()
            print("State -> SIT")
    elif state == "SIT":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")

        if time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Timeout. Reset."
            print("Timeout -> START")
        # 힙이 무릎보다 위 + 초기 힙 위치와 비슷
        elif hip_y < knee_y - 0.05 and (hip_y <= init_hip + 0.05):
            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            print(f"스쿼트 횟수: {squat_count}")
    elif state == "STAND":
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            feedback = "Standing: Good posture"
        else:
            feedback = "Standing: Straighten back"
        if hip_y >= knee_y - 0.05:
            state = "SIT"
            sit_start_time = time.time()
            feedback = "Transitioning to SIT."
            print("State -> SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [ ]:
cap = cv2.VideoCapture(0) # 실시간 웹캠 사용

# 전역 변수 초기화
squat_count = 0
initial_hip_y_dict = {"left": None, "right": None}
state = "START"
sit_start_time = None

active_side = None
side_stability_counter = 0

# FPS 측정용
prev_time = time.time()
frame_count = 0
fps = 0.0

# 세션 변수
session_running = False
session_start_time = None
last_display_frame = None

with mp_pose.Pose(
    model_complexity=1,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    if not session_running and last_display_frame is not None:
      freeze = last_display_frame.copy()
      cv2.imshow("MediaPipe Pose", resize_image(freeze, MAX_WIDTH))
      key = cv2.waitKey(1) & 0xFF
      if key in (27, ord('q')):
        break
      continue

    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break

    # BGR → RGB 변환
    frame.flags.writeable = False
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    feedback = ""  # 피드백 초기화
    extra_lines = []

    if session_running and results.pose_landmarks:
      landmarks = results.pose_landmarks.landmark
      active_side, side_stability_counter, left_sum, right_sum = check_side(
        landmarks, active_side, side_stability_counter
      )
      if active_side is not None:
        state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra_lines = process_state(landmarks, active_side, state, initial_hip_y_dict, sit_start_time, squat_count)
    elif not session_running:
      feedback = "stopped"

    frame.flags.writeable = True
    frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    if session_running and session_start_time:
      elapsed = time.time() - session_start_time
      cv2.putText(frame, f"Elapsed: {elapsed:5.1f}s / {SESSION_MAX_SECONDS}s",
                  (300, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,0), 2,
                  cv2.LINE_AA)
      if elapsed >= SESSION_MAX_SECONDS:
        session_running = False
        last_display_frame = frame.copy()

    # 피드백 및 상태 정보 표시
    draw_feedback(frame, feedback, state, squat_count, extra_lines)

    # FPS 계산 (0.5초마다 갱신)
    frame_count += 1
    now = time.time()
    if now - prev_time >= 0.5:
      fps = frame_count / (now - prev_time)
      prev_time = now
      frame_count = 0
    cv2.putText(frame, f"FPS: {fps:.1f}", (30, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

    if not session_running:
      cv2.putText(frame, "Press 's' to START (30s session)", (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)
    elif session_running:
      cv2.putText(frame, "Press 'e' to END",
                  (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)

    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(frame, MAX_WIDTH)

    if session_running:
      last_display_frame = frame.copy()

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    key = cv2.waitKey(1) & 0xFF
    if key in (27, ord('q')):
      break
    if key == ord('s') and not session_running:
      session_running = True
      session_start_time = time.time()
    if key == ord('e') and session_running:
      session_running = False
      last_display_frame = frame.copy()

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 1.436
Initial Hip Y: 1.055


### 실시간

- 왼쪽-오른쪽 구분 로직 추가
- 스쿼트 횟수로 중단

[x] 횟수 따른 결과 출력

[] 각 포지션에 따른 정확도 계산

- 각 포지션의 실수 퍼센트 계산 후 출력
- 등급(grade): =85: Perfect, >=70: Good, >=55: Fair, else: Poor

구현 
- SIT 상태 동안 bottom 후보(무릎 각도 최소 시점) 업데이트.
- SIT→STAND 전환 시 그동안 수집한 bottom metrics로 rep 점수 계산.
- rep_scores, rep_grades, avg_score 유지.
- 최근 rep 점수/평균 화면 표시.

[x] 자세가 안 좋아도 카운트는 하기

[] 처음 시작 시 시작함을 알리기

[x] 화면 크기 키우기

[x] 폰트 색 잘 보이는 색으로 통일하기

[x] 내장 카메라 / 캠
- 내장 카메라 0
- 외장 USB 웹캠 1

[] Sit -> Stand 조건 변경
- rising 상태 추가
- rising : 상승 중임을 확인
- stand : 상승 멈춤


In [20]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
# CAM_INDEX = 0  # 내장 카메라
CAM_INDEX = 1 # 외장 USB 웹캠
USE_DIRECTSHOW = True
RESTART_RESET = False
squat_videoPath = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'

MAX_WIDTH = 1000  # 영상 최대 너비
FEEDBACK_COLOR = (255, 255, 255)
EXTRA_COLOR    = (0, 170, 255)

TIME_LIMIT = 10  # 시간 제한 (초)
SIDE_VIS_MARGIN = 0.15          # visibility 합 차이 최소 마진
SIDE_STABILITY_FRAMES = 10      # 연속 프레임 유지 필요 수
MIN_VIS = 0.7 # 최소 visibility
# SESSION_MAX_SECONDS = 30 # 30초
session_running = False
session_start_time = None
last_display_frame = None
TARGET_REPS = 3 # 목표 스쿼트 반복 횟수
TARGET_REP_HOLD_FRAMES = 10 # 완료 후 유지 프레임 수
COUNTDOWN_SECONDS = 3 # 시작 카운트다운

# MIN standard
CALIB_MIN_VIS = 0.55
BACK_STRAIGHTNESS_THRESHOLD_CALIB = 0.18
VIS_EWMA_ALPHA = 0.4
vis_ewma = {
    "left": {"HIP": None, "KNEE": None, "SHOULDER": None},
    "right": {"HIP": None, "KNEE": None, "SHOULDER": None}
}

BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준
# KNEE_MIN_OK = 70
# KNEE_MAX_OK = 110
# BACK_DEV_TOL = 0.18
# IDEAL_DEPTH_RATIO = 1.0
# TARGET_SIT_FRAMES = 6
# W_KNEE = 0.35
# W_BACK = 0.25
# W_DEPTH = 0.25
# W_CTRL = 0.15
# MIN_DEPTH_DELTA = 0.08
# MIN_SIT_FRAMES_FOR_DEPTH = 5      # SIT에서 최소 유지 프레임
# UP_FRAMES_CONFIRM = 3
# HIP_ABOVE_KNEE_GAP = 0.05
# INIT_HIP_MARGIN = 0.05
# VEL_WINDOW = 5
# MIN_UP_VEL = 0.003

# rep_scores = []
# rep_grades = []
# rep_breakdowns = []
# recent_rep_feedback = ""
# avg_score = 0.0

ENTER_SIT_GAP = 0.06
DEPTH_MIN = 0.07
BOTTOM_MIN_FRAMES = 4
VEL_WINDOW = 4
UP_VEL_THRESH = -0.0025
DOWN_VEL_THRESH = 0.0015
UP_MIN_FRAMES = 10 # up
KNEE_GAP = 0.07 # up
STAND_HIP_MARGIN = 0.03 # down
GRACE_FRAMES = 5
SLOW_VEL = -0.0005
MIN_UP_DISP = 0.04

calib_good_frames = 0
calib_hip_samples = []
calibrated = False

track = {
    "hip_init": None, # 초기 힙 위치
    "hip_bottom": None, # 가장 낮은 힙 위치
    "knee_bottom": None, # 가장 낮은 무릎 위치
    "sit_frames": 0, # SIT 프레임 수
    "bottom_locked": False, # SIT에서 바닥 확인
    "rise_frames": 0, # 상승 프레임 수
    "hip_history": [] # 최근 힙 위치 기록
}

SIDE_IDX = {
    "left": {
        "HIP": mp_pose.PoseLandmark.LEFT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.LEFT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.LEFT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.LEFT_SHOULDER.value
    },
    "right": {
        "HIP": mp_pose.PoseLandmark.RIGHT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.RIGHT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    }
}

In [21]:
# def joints_visible(landmarks, side, min_vis=MIN_VIS):
#     ids = SIDE_IDX[side]
#     needed = ["HIP", "KNEE", "ANKLE", "SHOULDER"]
#     # needed = ["HIP", "KNEE", "SHOULDER"]
#     vis_map = {name: landmarks[ids[name]].visibility for name in needed}
#     ok = all(vis_map[n] >= min_vis for n in needed)
#     return ok, vis_map

# # joints_visible 교체 (각 관절 visibility 반환)
# def joints_visible(landmarks, side, min_vis=MIN_VIS):
#     ids = SIDE_IDX[side]
#     needed = ["HIP","KNEE","SHOULDER"]  # ANKLE 제외 (원하면 추가)
#     vis_map = {n: landmarks[ids[n]].visibility for n in needed}
#     low_map = {k:v for k,v in vis_map.items() if v < min_vis}
#     ok = len(low_map) == 0
#     return ok, vis_map, low_map

# joints_visible 보강 (EWMA 적용 + 캘리브 모드 구분)
def joints_visible(landmarks, side, min_vis, use_ewma=True):
    ids = SIDE_IDX[side]
    names = ["HIP","KNEE","SHOULDER"]
    raw = {n: landmarks[ids[n]].visibility for n in names}

    if use_ewma:
        for n in names:
            prev = vis_ewma[side][n]
            v = raw[n]
            vis_ewma[side][n] = v if prev is None else (VIS_EWMA_ALPHA * v + (1-VIS_EWMA_ALPHA)*prev)
        smoothed = {n: vis_ewma[side][n] for n in names}
    else:
        smoothed = raw

    # 무릎은 강제 조건, 나머지는 참고 (무릎 실패면 즉시 FAIL)
    knee_ok = smoothed["KNEE"] >= min_vis
    hip_ok = smoothed["HIP"] >= (min_vis - 0.05)
    shoulder_ok = smoothed["SHOULDER"] >= (min_vis - 0.05)

    ok = knee_ok and hip_ok and shoulder_ok
    low_map = {k: smoothed[k] for k in names if (k=="KNEE" and not knee_ok) or
               (k!="KNEE" and smoothed[k] < (min_vis - 0.05))}
    return ok, smoothed, low_map

In [4]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [5]:
def check_side(landmarks, prev_side, stability_counter):
    l_ids = SIDE_IDX["left"]
    r_ids = SIDE_IDX["right"]
    l_sum = (landmarks[l_ids["HIP"]].visibility +
             landmarks[l_ids["KNEE"]].visibility +
             landmarks[l_ids["ANKLE"]].visibility)
    r_sum = (landmarks[r_ids["HIP"]].visibility +
             landmarks[r_ids["KNEE"]].visibility +
             landmarks[r_ids["ANKLE"]].visibility)

    candidate = prev_side
    diff = l_sum - r_sum
    if prev_side is None:
        if l_sum >= r_sum:
            candidate = "left"
        else:
            candidate = "right"
        stability_counter = 0
    else:
        if diff > SIDE_VIS_MARGIN and prev_side != "left":
            candidate = "left"
        elif diff < -SIDE_VIS_MARGIN and prev_side != "right":
            candidate = "right"
        else:
            return prev_side, stability_counter, l_sum, r_sum

        if candidate != prev_side:
            stability_counter += 1
            if stability_counter >= SIDE_STABILITY_FRAMES:
                prev_side = candidate
                stability_counter = 0
            else:
                candidate = prev_side
        else:
            stability_counter = 0

    return candidate, stability_counter, l_sum, r_sum

In [6]:
def check_knee_angle(landmarks, side):
    ids = SIDE_IDX[side]

    hip = [landmarks[ids["HIP"]].x, landmarks[ids["HIP"]].y]
    knee = [landmarks[ids["KNEE"]].x, landmarks[ids["KNEE"]].y]
    ankle = [landmarks[ids["ANKLE"]].x, landmarks[ids["ANKLE"]].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(hip, knee, ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        # is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        # is_bent_correctly = False

    # return feedback, is_bent_correctly
    return feedback, knee_angle

In [7]:
def classify_grade(score: float) -> str:
    if score >= 85: return "Perfect"
    if score >= 70: return "Good"
    if score >= 55: return "Fair"
    return "Poor"

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

In [8]:
def compute_rep_score(track: dict):
    if (track["knee_angle_min"] is None or
        track["hip_init"] is None or
        track["hip_bottom"] is None or
        track["knee_bottom"] is None):
        return 0.0, {"knee":0,"back":0,"depth":0,"control":0}, "Invalid"

    knee_angle = track["knee_angle_min"]
    back_dev = track["back_dev_max"]
    hip_init = track["hip_init"]
    depth_raw = track["hip_bottom"] - hip_init
    knee_span = track["knee_bottom"] - hip_init if track["knee_bottom"] > hip_init else None
    if not knee_span or knee_span <= 1e-6:
        depth_ratio = 0
    else:
        depth_ratio = depth_raw / knee_span

    knee_norm = clamp((KNEE_MAX_OK - knee_angle)/(KNEE_MAX_OK - KNEE_MIN_OK), 0, 1)
    back_norm = clamp(1 - (back_dev / BACK_DEV_TOL), 0, 1)
    depth_norm = clamp(1 - abs(depth_ratio - IDEAL_DEPTH_RATIO)/IDEAL_DEPTH_RATIO, 0, 1)
    ctrl_norm = clamp(track["sit_frames"] / TARGET_SIT_FRAMES, 0, 1)

    score = 100 * (W_KNEE*knee_norm + W_BACK*back_norm + W_DEPTH*depth_norm + W_CTRL*ctrl_norm)
    grade = classify_grade(score)
    breakdown = {
        "knee": round(knee_norm*100,1),
        "back": round(back_norm*100,1),
        "depth": round(depth_norm*100,1),
        "control": round(ctrl_norm*100,1),
        "knee_angle": round(knee_angle,1),
        "depth_ratio": round(depth_ratio,2),
        "back_dev": round(back_dev,3),
        "sit_frames": track["sit_frames"]
    }
    return round(score,1), breakdown, grade

In [9]:
def put_text_with_bg(img, text, org, color, font=cv2.FONT_HERSHEY_SIMPLEX,
                     scale=0.8, thickness=2, bg_color=(30,30,30), alpha=0.6, pad=6):
    (tw, th), bl = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    # 배경 영역
    bg_tl = (x - pad, y - th - pad)
    bg_br = (x + tw + pad, y + bl + pad//2)
    # 반투명 배경
    overlay = img.copy()
    cv2.rectangle(overlay, bg_tl, bg_br, bg_color, -1)
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
    # 텍스트
    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)

In [10]:
def draw_feedback(image, feedback, state, squat_count, extra_lines=None):
    put_text_with_bg(
        image,
        f"State: {state} | Count: {squat_count}",
        (30, 50),
        (0, 255, 0),  # 상태/카운트는 녹색 유지
        scale=0.9
    )
    if feedback:
        put_text_with_bg(
            image,
            feedback,
            (30, 95),
            FEEDBACK_COLOR,
            scale=0.85
        )
    if extra_lines:
        y = 140
        for line in extra_lines:
            put_text_with_bg(
                image,
                line,
                (30, y),
                EXTRA_COLOR,
                scale=0.65
            )
            y += 38

In [11]:
def resize_image(image, max_width):
    h, w = image.shape[:2]
    if w == max_width:
        return image
    scale = max_width / w
    return cv2.resize(image, (max_width, int(h * scale)), interpolation=cv2.INTER_LINEAR)

In [12]:
# --- CALIB 디버그 상태 추적 전역 추가 (상수 구역 아래) ---
calib_debug_prev = {
    "visible": None,
    "ok_back": None,
    "ok_gap": None,
    "frames": -1
}

In [23]:
def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
    global calib_good_frames, calib_hip_samples, calibrated, track, calib_debug_prev

    ids = SIDE_IDX[side]
    hip_y = landmarks[ids["HIP"]].y
    knee_y = landmarks[ids["KNEE"]].y
    shoulder_y = landmarks[ids["SHOULDER"]].y

    # 1) visibility 단 한 번만 계산
    vis_ok, vis_map, low_map = joints_visible(landmarks, side)

    # 2) CALIB 상태 처리
    if state == "CALIB":
        extra = []
        vis_ok, vis_map, low_map = joints_visible(landmarks, side, CALIB_MIN_VIS, use_ewma=True)
        if not vis_ok:
            # 변화 시만 로그
            low_txt = ", ".join(f"{k}:{low_map[k]:.2f}" for k in low_map)
            print(f"[CALIB] VIS FAIL -> {low_txt} (thr {MIN_VIS})")
            feedback = "Improve visibility"
            extra.append("VIS:FAIL")
            extra.extend([f"{k}:{vis_map[k]:.2f}" for k in vis_map])
            calib_good_frames = 0
            calib_hip_samples.clear()
            return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

        # 자세 조건
        back_straightness = abs(hip_y - shoulder_y)
        knee_hip_gap = knee_y - hip_y
        ok_back = back_straightness < BACK_STRAIGHTNESS_THRESHOLD
        ok_gap  = knee_hip_gap >= 0.10

        if ok_back and ok_gap:
            calib_good_frames += 1
            calib_hip_samples.append(hip_y)
        else:
            if calib_good_frames > 0:
                print(f"[CALIB] RESET back_ok={ok_back} gap_ok={ok_gap} back={back_straightness:.3f} gap={knee_hip_gap:.3f}")
            calib_good_frames = 0
            calib_hip_samples.clear()

        # 상태 변화 로깅
        changed = (
            calib_debug_prev.get("vis_ok") != vis_ok or
            calib_debug_prev.get("ok_back") != ok_back or
            calib_debug_prev.get("ok_gap") != ok_gap or
            calib_debug_prev.get("frames") != calib_good_frames
        )
        if changed:
            print("[CALIB] "
                  f"VIS:OK | BACK:{'OK' if ok_back else 'FAIL'}({back_straightness:.3f}<{BACK_STRAIGHTNESS_THRESHOLD}) | "
                  f"GAP:{'OK' if ok_gap else 'FAIL'}({knee_hip_gap:.3f}>=0.10) | FRAMES:{calib_good_frames}/15")
            calib_debug_prev.update({
                "vis_ok": vis_ok,
                "ok_back": ok_back,
                "ok_gap": ok_gap,
                "frames": calib_good_frames
            })

        print(f"[CALIB] FR:{calib_good_frames}/15 VIS:OK KNEE:{vis_map['KNEE']:.2f} BACK:{back_straightness:.3f}<{BACK_STRAIGHTNESS_THRESHOLD_CALIB}={ok_back} GAP:{knee_hip_gap:.3f}>=0.10={ok_gap}")

        feedback = "Stand straight for calibration"
        extra.append(f"KNEEvis:{vis_map['KNEE']:.2f}")
        extra.append(f"BACK:{'OK' if ok_back else 'FAIL'} {back_straightness:.3f}")
        extra.append(f"GAP:{'OK' if ok_gap else 'FAIL'} {knee_hip_gap:.3f}")
        extra.append(f"Frames {calib_good_frames}/15")

        if calib_good_frames >= 15:
            avg_hip = sum(calib_hip_samples)/len(calib_hip_samples)
            initial_hip_y_dict[side] = avg_hip
            track["hip_init"] = avg_hip
            calibrated = True
            calib_good_frames = 0
            calib_hip_samples.clear()
            feedback = "Calibrated! Start ready"
            state = "START"
            print(f"[CALIBRATION] hip_init set to {avg_hip:.3f}")
        return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

    # 3) CALIB 이후: visibility 실패 시 즉시 반환
    if not vis_ok:
        feedback = "Low visibility - hold still"
        extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]
        return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

    # 4) hip_init 1회 확보
    if track["hip_init"] is None:
        if initial_hip_y_dict[side] is not None:
            track["hip_init"] = initial_hip_y_dict[side]
        else:
            initial_hip_y_dict[side] = hip_y
            track["hip_init"] = hip_y
            print(f"Fallback hip_init set {hip_y:.3f}")
    init_hip = track["hip_init"]

    # 5) 속도 계산
    track["hip_history"].append(hip_y)
    if len(track["hip_history"]) > VEL_WINDOW:
        track["hip_history"].pop(0)
    if len(track["hip_history"]) >= 2:
        avg_vel = (track["hip_history"][-1] - track["hip_history"][0]) / (len(track["hip_history"]) - 1)
    else:
        avg_vel = 0.0

    back_straightness = abs(hip_y - shoulder_y)
    feedback = ""
    extra = []

    # START
    if state == "START":
        feedback = "Check posture..."
        extra.append("Back: Good" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Back: Straighten!")
        if abs(hip_y - knee_y) < ENTER_SIT_GAP:
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y]
            })
            print("START -> SIT")

    elif state == "SIT":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")
        if hip_y > track["hip_bottom"]:
            track["hip_bottom"] = hip_y
            track["knee_bottom"] = knee_y
        track["sit_frames"] += 1
        depth = track["hip_bottom"] - init_hip
        if (not track["bottom_locked"] and
            track["sit_frames"] >= BOTTOM_MIN_FRAMES and
            depth >= DEPTH_MIN):
            track["bottom_locked"] = True
            extra.append("Bottom OK")

        if track["bottom_locked"] and avg_vel < UP_VEL_THRESH:
            state = "RISING"
            track["rise_frames"] = 0
            track["hip_history"] = [hip_y]
            print("SIT -> RISING")
        elif time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Timeout. Reset."
            print("SIT -> START (timeout)")

    elif state == "RISING":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")
        stand_ready = (
            hip_y < knee_y - KNEE_GAP and
            hip_y <= init_hip + STAND_HIP_MARGIN and
            track["rise_frames"] >= UP_MIN_FRAMES
        )
        if stand_ready:
            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            track["hip_history"] = [hip_y]
            print("RISING -> STAND")
            return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

        if avg_vel < UP_VEL_THRESH:
            track["rise_frames"] += 1
            feedback = "Rising..."
        else:
            # 불완전 카운트 (원치 않으면 이 블록 제거)
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            extra.append("Incomplete")
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y]
            })
            print("RISING -> SIT (incomplete)")
            return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra
        extra.append(f"riseF:{track['rise_frames']}")
        extra.append(f"vel:{avg_vel:.4f}")

    elif state == "STAND":
        feedback = "Standing: Good posture" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Standing: Straighten back"
        if avg_vel > abs(UP_VEL_THRESH) and hip_y >= knee_y - ENTER_SIT_GAP:
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y]
            })
            feedback = "Descend"
            print("STAND -> SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [14]:
# def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
#     global calib_good_frames, calib_hip_samples, calibrated, track

#     ids = SIDE_IDX[side]
#     hip_y = landmarks[ids["HIP"]].y
#     knee_y = landmarks[ids["KNEE"]].y
#     shoulder_y = landmarks[ids["SHOULDER"]].y

#     visible_ok, vis_map = joints_visible(landmarks, side)

#     if not visible_ok:
#         # 스쿼트 판정/카운트 건너뜀
#         feedback = "Low visibility - hold still"
#         extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]
#         return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

#     if state == "CALIB":
#         extra = []
#         back_straightness = abs(hip_y - shoulder_y)
#         knee_hip_gap = knee_y - hip_y
#         ok_back = back_straightness < BACK_STRAIGHTNESS_THRESHOLD
#         ok_gap = knee_hip_gap >= 0.10

#         if ok_back and ok_gap:
#             calib_good_frames += 1
#             calib_hip_samples.append(hip_y)
#         else:
#             calib_good_frames = 0
#             calib_hip_samples.clear()

#         feedback = "Stand straight for calibration"

#         if calib_good_frames >= 15:
#             avg_hip = sum(calib_hip_samples)/len(calib_hip_samples)
#             initial_hip_y_dict[side] = avg_hip
#             track["hip_init"] = avg_hip
#             calibrated = True
#             calib_good_frames = 0
#             calib_hip_samples.clear()
#             feedback = "Calibrated! Start ready"
#             state = "START"
#             print(f"[CALIBRATION] hip_init set to {avg_hip:.3f}")
#         return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

#     # 처음 자세의 엉덩이 위치 저장
#     if track["hip_init"] is None:
#         if initial_hip_y_dict[side] is not None:
#             track["hip_init"] = initial_hip_y_dict[side]
#         else:
#             initial_hip_y_dict[side] = hip_y
#             track["hip_init"] = hip_y
#             print(f"Fallback hip_init set {hip_y:.3f}")
#     init_hip = track["hip_init"]

#     # 속도용 힙 히스토리 (프레임 단위 1회)
#     # RISING 진입 직후 프레임에서는 이전 상태에서 리셋됨
#     track["hip_history"].append(hip_y)
#     if len(track["hip_history"]) > VEL_WINDOW:
#         track["hip_history"].pop(0)
#     if len(track["hip_history"]) >= 2:
#         avg_vel = (track["hip_history"][-1] - track["hip_history"][0]) / (len(track["hip_history"])-1)
#     else:
#         avg_vel = 0.0

#     # hh = track["hip_history"]
#     # hh.append(hip_y)
#     # if len(hh) > VEL_WINDOW:
#     #     hh.pop(0)
#     # avg_vel = 0.0
#     # if len(hh) >= 2:
#     #     avg_vel = (hh[-1] - hh[0]) / (len(hh)-1)

#     # 허리 펴짐 정도 계산
#     back_straightness = abs(hip_y - shoulder_y)

#     feedback = ""
#     extra = []

#     # 상태 전환 로직
#     if state == "START":
#         feedback = "Check posture..."
#         if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
#             extra.append("Back: Good")
#         else:
#             extra.append("Back: Straighten!")
#         # 엉덩이 :=: 무릎
#         if abs(hip_y - knee_y) < ENTER_SIT_GAP:
#             state = "SIT"
#             sit_start_time = time.time()
#             track.update({
#                 "hip_bottom": hip_y,
#                 "knee_bottom": knee_y,
#                 "sit_frames": 0,
#                 "bottom_locked": False,
#                 "rise_frames": 0,
#                 "hip_history": [hip_y]
#             })
#             print("State -> SIT")
#     elif state == "SIT":
#         feedback, knee_angle = check_knee_angle(landmarks, side)
#         extra.append(f"KneeAngle: {knee_angle:.1f}")
#         # bottom 갱신
#         if hip_y > track["hip_bottom"]:
#             track["hip_bottom"] = hip_y
#             track["knee_bottom"] = knee_y
#         track["sit_frames"] += 1

#         depth = track["hip_bottom"] - init_hip
#         if (not track["bottom_locked"]
#             and track["sit_frames"] >= BOTTOM_MIN_FRAMES
#             and depth >= DEPTH_MIN):
#             track["bottom_locked"] = True
#             extra.append("Bottom OK")

#         # 상승 감지 → RISING
#         if track["bottom_locked"] and avg_vel < UP_VEL_THRESH:
#             state = "RISING"
#             track["rise_frames"] = 0
#             track["hip_history"] = [hip_y]
#             print("SIT -> RISING")
#         elif time.time() - sit_start_time > TIME_LIMIT:
#             state = "START"
#             feedback = "Timeout. Reset."
#             print("Timeout: SIT -> START")

#     elif state == "RISING":
#         feedback, knee_angle = check_knee_angle(landmarks, side)
#         extra.append(f"KneeAngle: {knee_angle:.1f}")

#         stand_ready = (
#             hip_y < knee_y - KNEE_GAP and
#             hip_y <= init_hip + STAND_HIP_MARGIN and
#             track["rise_frames"] >= UP_MIN_FRAMES
#         )
#         if stand_ready:
#             state = "STAND"
#             squat_count += 1
#             feedback = f"Squat Count: {squat_count}"
#             track["hip_history"] = [hip_y]
#             print("RISING -> STAND (good)")
#             return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

#         # 2) 상승 속도 판단
#         if avg_vel < UP_VEL_THRESH:          # 계속 상승 중
#             track["rise_frames"] += 1
#             feedback = "Rising..."
#         else:
#             # 상승 멈춤 / 느려짐 → 불완전 카운트 후 SIT 복귀
#             squat_count += 1
#             feedback = f"Squat Count: {squat_count}"
#             extra.append("Incomplete")
#             state = "SIT"
#             sit_start_time = time.time()
#             track.update({
#                 "hip_bottom": hip_y,
#                 "knee_bottom": knee_y,
#                 "sit_frames": 0,
#                 "bottom_locked": False,
#                 "rise_frames": 0,
#                 "hip_history": [hip_y]
#             })
#             print("RISING -> SIT (incomplete)")
#             return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra


#         # if avg_vel < UP_VEL_THRESH: # 상승 유지
#         #     track["rise_frames"] += 1
#         # # else:
#         # #     extra.append("Plateau")
#         # else: # 상승 멈춤
#         #     if not (hip_y < knee_y - KNEE_GAP and hip_y <= init_hip + STAND_HIP_MARGIN):
#         #         squat_count += 1
#         #         feedback = f"Squat Count: {squat_count}"
#         #         state = "SIT"
#         #         sit_start_time = time.time()
#         #         track.update({
#         #             "hip_bottom": hip_y,
#         #             "knee_bottom": knee_y,
#         #             "sit_frames": 0,
#         #             "bottom_locked": False,
#         #             "rise_frames": 0,
#         #             "hip_history": [hip_y]
#         #         })
#         #         print("RISING -> SIT")
#         #         return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

#         # # 상승 완료 확인
#         # # 힙이 무릎보다 위 + 초기 힙 근처 + 연속 상승 프레임 확인
#         # if  (hip_y < knee_y - KNEE_GAP
#         #      and hip_y <= init_hip + STAND_HIP_MARGIN
#         #      and track["rise_frames"] >= UP_MIN_FRAMES):
#         #     state = "STAND"
#         #     print("RISING -> STAND")
#         #     squat_count += 1
#         #     feedback = f"Squat Count: {squat_count}"
#         #     track["hip_history"] = [hip_y]
#     elif state == "STAND":
#         if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
#             feedback = "Standing: Good posture"
#         else:
#             feedback = "Standing: Straighten back"
#         if avg_vel > abs(UP_VEL_THRESH) and hip_y >= knee_y - ENTER_SIT_GAP: # 값 증가(하강)
#             state = "SIT"
#             sit_start_time = time.time()
#             track.update({
#                 "hip_bottom": hip_y,
#                 "knee_bottom": knee_y,
#                 "sit_frames": 0,
#                 "bottom_locked": False,
#                 "rise_frames": 0,
#                 "hip_history": [hip_y]
#             })
#             feedback = "Transitioning to SIT."
#             print("State -> SIT")

#     return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [18]:
# ...existing code...

def draw_stand_guides(frame, track, side, landmarks):
    """
    초기 캘리브 hip_init 과 STAND 판정에 필요한 높이들을 시각화.
    y는 (0=상단, 1=하단). hip_y 작아질수록 위로 올라온 것.
    """
    if track.get("hip_init") is None:
        return
    H, W = frame.shape[:2]
    ids = SIDE_IDX[side]
    hip_y_now = landmarks[ids["HIP"]].y
    knee_y_now = landmarks[ids["KNEE"]].y

    hip_init = track["hip_init"]
    stand_margin_y = hip_init + STAND_HIP_MARGIN            # 허용 높이(초기보다 약간 아래까지 허용)
    knee_req_y = knee_y_now - KNEE_GAP                      # 무릎 대비 조건
    target_y = min(stand_margin_y, knee_req_y)              # 두 조건 모두 만족해야 하므로 더 작은 y (위쪽)

    # 픽셀 변환
    hip_init_px = int(hip_init * H)
    stand_margin_px = int(stand_margin_y * H)
    knee_req_px = int(knee_req_y * H)
    target_px = int(target_y * H)
    hip_now_px = int(hip_y_now * H)

    # 선 그리기
    # 초기 힙 (흰색)
    cv2.line(frame, (0, hip_init_px), (W, hip_init_px), (255,255,255), 1)
    cv2.putText(frame, "hip_init", (10, hip_init_px-5), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1, cv2.LINE_AA)

    # stand margin (청록)
    cv2.line(frame, (0, stand_margin_px), (W, stand_margin_px), (255,255,0), 1)
    cv2.putText(frame, "+margin", (10, stand_margin_px-5), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,0), 1, cv2.LINE_AA)

    # knee 조건선 (노랑)
    cv2.line(frame, (0, knee_req_px), (W, knee_req_px), (0,255,255), 1)
    cv2.putText(frame, "kneeGap", (10, knee_req_px-5), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,255), 1, cv2.LINE_AA)

    # 최종 목표(target) (초록 굵게)
    cv2.line(frame, (0, target_px), (W, target_px), (0,200,0), 2)
    cv2.putText(frame, "STAND target", (10, target_px-7), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,200,0), 1, cv2.LINE_AA)

    # 현재 HIP 점
    color = (0,200,0) if hip_y_now <= target_y else (0,0,255)
    cv2.circle(frame, (int(landmarks[ids["HIP"]].x * W), hip_now_px), 7, color, -1)

    # 남은 거리 (노말라이즈 y 차이 / 픽셀)
    remaining = max(0.0, hip_y_now - target_y)
    cv2.putText(frame, f"remain:{remaining:.3f}", (int(W*0.65), target_px-6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,200,0), 1, cv2.LINE_AA)

In [24]:
cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW if USE_DIRECTSHOW else 0)
# cap = cv2.VideoCapture(squat_videoPath)

# 전역 변수 초기화
squat_count = 0
initial_hip_y_dict = {"left": None, "right": None}
state = "CALIB"
sit_start_time = None

active_side = None
side_stability_counter = 0

# FPS 측정용
prev_time = time.time()
frame_count = 0
fps = 0.0

# 세션 변수
session_running = False
session_start_time = None
last_display_frame = None

# 시작 관련
training_active = False
ready_frames = 0
countdown_start = None #  카운트다운 시작 시간

# squat goal
target_reached = False
stand_hold = 0

with mp_pose.Pose(
    model_complexity=1,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    if not session_running and last_display_frame is not None and state == "FINISHED":
      freeze = last_display_frame.copy()
      cv2.putText(freeze, f"Completed {TARGET_REPS} Reps!",
                    (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2, cv2.LINE_AA)
      cv2.putText(freeze, "s : restart, q : quit",
                  (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2, cv2.LINE_AA)
      cv2.imshow("MediaPipe Pose", resize_image(freeze, MAX_WIDTH))

      key = cv2.waitKey(1) & 0xFF
      if key in (27, ord('q')):
        break
      if key == ord('s'):
        squat_count = 0
        target_reached = False
        stand_hold = 0
        state = "START"
        session_running = True
        session_start_time = time.time()
        last_display_frame = None
        stand_hold = 0
        print("[Session] RESTART")
      continue

    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break

    # BGR → RGB 변환
    frame.flags.writeable = False
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    feedback = ""  # 피드백 초기화
    extra_lines = []

    if session_running and results.pose_landmarks:
      landmarks = results.pose_landmarks.landmark
      active_side, side_stability_counter, left_sum, right_sum = check_side(
        landmarks, active_side, side_stability_counter
      )

      if active_side is not None:
        state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra_lines = process_state(landmarks, active_side, state, initial_hip_y_dict, sit_start_time, squat_count)

      if squat_count >= TARGET_REPS:
        if not target_reached:
          target_reached = True
          stand_hold = 0
        if state == "STAND":
          stand_hold += 1
        else:
          stand_hold = 0
        extra_lines.append(f"FinishHold: {stand_hold}/{TARGET_REP_HOLD_FRAMES}")
        feedback = f"Target Reps ({TARGET_REPS}) achieved!"
        if stand_hold >= TARGET_REP_HOLD_FRAMES:
          session_running = False
          state = "FINISHED"
          last_display_frame = frame.copy()
    elif not session_running:
      feedback = "stopped"

    frame.flags.writeable = True
    frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    # 피드백 및 상태 정보 표시
    draw_feedback(frame, feedback, state, squat_count, extra_lines)
    if session_running and results.pose_landmarks and active_side and 'track' in globals():
        draw_stand_guides(frame, track, active_side, results.pose_landmarks.landmark)
    # if recent_rep_feedback:
    #   put_text_with_bg(frame, recent_rep_feedback, (30, frame.shape[0]-150), (255,255,255), scale=0.7)
    #   put_text_with_bg(frame, f"Avg Score: {avg_score}", (30, frame.shape[0]-115), (0,255,255), scale=0.65)

    # FPS 계산 (0.5초마다 갱신)
    frame_count += 1
    now = time.time()
    if now - prev_time >= 0.5:
      fps = frame_count / (now - prev_time)
      prev_time = now
      frame_count = 0
    cv2.putText(frame, f"FPS: {fps:.1f}", (30, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

    if not session_running and state != "FINISHED":
      cv2.putText(frame, "s : start, goal : {TARGET_REPS}", (30, frame.shape[0]-55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (30, 180, 255), 2, cv2.LINE_AA)
    else:
      cv2.putText(frame, "e : quit",
                  (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (30, 180, 255), 2, cv2.LINE_AA)

    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(frame, MAX_WIDTH)

    if session_running:
      last_display_frame = frame.copy()

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)

    key = cv2.waitKey(1) & 0xFF
    if key in (27, ord('q')):
      break
    if key == ord('s') and not session_running:
      session_running = True
      session_start_time = time.time()
    if key == ord('e') and session_running:
      session_running = False
      last_display_frame = frame.copy()

cap.release()
cv2.destroyAllWindows()

TypeError: joints_visible() missing 1 required positional argument: 'min_vis'

### 실시간 - calib 없음

- 왼쪽-오른쪽 구분 로직 추가
- 스쿼트 횟수로 중단

[x] 횟수 따른 결과 출력

[] 각 포지션에 따른 정확도 계산

- 각 포지션의 실수 퍼센트 계산 후 출력
- 등급(grade): =85: Perfect, >=70: Good, >=55: Fair, else: Poor

구현 
- SIT 상태 동안 bottom 후보(무릎 각도 최소 시점) 업데이트.
- SIT→STAND 전환 시 그동안 수집한 bottom metrics로 rep 점수 계산.
- rep_scores, rep_grades, avg_score 유지.
- 최근 rep 점수/평균 화면 표시.

[x] 자세가 안 좋아도 카운트는 하기

[] 처음 시작 시 시작함을 알리기

[x] 화면 크기 키우기

[x] 폰트 색 잘 보이는 색으로 통일하기

[x] 내장 카메라 / 캠
- 내장 카메라 0
- 외장 USB 웹캠 1

[] Sit -> Stand 조건 변경
- rising 상태 추가
- rising : 상승 중임을 확인
- stand : 상승 멈춤


In [ ]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
# CAM_INDEX = 0  # 내장 카메라
CAM_INDEX = 1 # 외장 USB 웹캠
USE_DIRECTSHOW = True
RESTART_RESET = False
squat_videoPath = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'

MAX_WIDTH = 1000  # 영상 최대 너비
FEEDBACK_COLOR = (255, 255, 255)
EXTRA_COLOR    = (0, 170, 255)

TIME_LIMIT = 10  # 시간 제한 (초)
SIDE_VIS_MARGIN = 0.15          # visibility 합 차이 최소 마진
SIDE_STABILITY_FRAMES = 10      # 연속 프레임 유지 필요 수
MIN_VIS = 0.7 # 최소 visibility
# SESSION_MAX_SECONDS = 30 # 30초
session_running = False
session_start_time = None
last_display_frame = None
TARGET_REPS = 3 # 목표 스쿼트 반복 횟수
TARGET_REP_HOLD_FRAMES = 3 # 완료 후 유지 프레임 수
COUNTDOWN_SECONDS = 3 # 시작 카운트다운

# MIN standard
CALIB_MIN_VIS = 0.55
BACK_STRAIGHTNESS_THRESHOLD_CALIB = 0.18
VIS_EWMA_ALPHA = 0.4
vis_ewma = {
    "left": {"HIP": None, "KNEE": None, "SHOULDER": None},
    "right": {"HIP": None, "KNEE": None, "SHOULDER": None}
}

BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준
KNEE_MIN_OK = 70
KNEE_MAX_OK = 110
BACK_DEV_TOL = 0.18
IDEAL_DEPTH_RATIO = 1.0
TARGET_SIT_FRAMES = 6
W_KNEE = 0.35
W_BACK = 0.25
W_DEPTH = 0.25
W_CTRL = 0.15
# MIN_DEPTH_DELTA = 0.08
# MIN_SIT_FRAMES_FOR_DEPTH = 5      # SIT에서 최소 유지 프레임
# UP_FRAMES_CONFIRM = 3
# HIP_ABOVE_KNEE_GAP = 0.05
# INIT_HIP_MARGIN = 0.05
# VEL_WINDOW = 5
# MIN_UP_VEL = 0.003

rep_scores = []
rep_grades = []
rep_breakdowns = []
recent_rep_feedback = ""
avg_score = 0.0

ENTER_SIT_GAP = 0.06
DEPTH_MIN = 0.07
BOTTOM_MIN_FRAMES = 4
VEL_WINDOW = 4
UP_VEL_THRESH = -0.0025
DOWN_VEL_THRESH = 0.0015
UP_MIN_FRAMES = 10 # up
KNEE_GAP = 0.07 # up
STAND_HIP_MARGIN = 0.03 # down
GRACE_FRAMES = 5
SLOW_VEL = -0.0005
MIN_UP_DISP = 0.04
STAND_PROGRESS = 0.90

calib_good_frames = 0
calib_hip_samples = []
calibrated = False

track = {
    "hip_init": None, # 초기 힙 위치
    "hip_bottom": None, # 가장 낮은 힙 위치
    "knee_bottom": None, # 가장 낮은 무릎 위치
    "sit_frames": 0, # SIT 프레임 수
    "bottom_locked": False, # SIT에서 바닥 확인
    "rise_frames": 0, # 상승 프레임 수
    "hip_history": [], # 최근 힙 위치 기록
    "knee_angle_min": None,
    "back_dev_max": 0.0,
    "scored": False
}

SIDE_IDX = {
    "left": {
        "HIP": mp_pose.PoseLandmark.LEFT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.LEFT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.LEFT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.LEFT_SHOULDER.value
    },
    "right": {
        "HIP": mp_pose.PoseLandmark.RIGHT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.RIGHT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    }
}

In [29]:
def joints_visible(landmarks, side, min_vis=MIN_VIS):
    ids = SIDE_IDX[side]
    # needed = ["HIP", "KNEE", "ANKLE", "SHOULDER"]
    needed = ["HIP", "KNEE", "SHOULDER"]
    vis_map = {name: landmarks[ids[name]].visibility for name in needed}
    ok = all(vis_map[n] >= min_vis for n in needed)
    return ok, vis_map

In [30]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [31]:
def check_side(landmarks, prev_side, stability_counter):
    l_ids = SIDE_IDX["left"]
    r_ids = SIDE_IDX["right"]
    l_sum = (landmarks[l_ids["HIP"]].visibility +
             landmarks[l_ids["KNEE"]].visibility +
             landmarks[l_ids["ANKLE"]].visibility)
    r_sum = (landmarks[r_ids["HIP"]].visibility +
             landmarks[r_ids["KNEE"]].visibility +
             landmarks[r_ids["ANKLE"]].visibility)

    candidate = prev_side
    diff = l_sum - r_sum
    if prev_side is None:
        if l_sum >= r_sum:
            candidate = "left"
        else:
            candidate = "right"
        stability_counter = 0
    else:
        if diff > SIDE_VIS_MARGIN and prev_side != "left":
            candidate = "left"
        elif diff < -SIDE_VIS_MARGIN and prev_side != "right":
            candidate = "right"
        else:
            return prev_side, stability_counter, l_sum, r_sum

        if candidate != prev_side:
            stability_counter += 1
            if stability_counter >= SIDE_STABILITY_FRAMES:
                prev_side = candidate
                stability_counter = 0
            else:
                candidate = prev_side
        else:
            stability_counter = 0

    return candidate, stability_counter, l_sum, r_sum

In [32]:
def check_knee_angle(landmarks, side):
    ids = SIDE_IDX[side]

    hip = [landmarks[ids["HIP"]].x, landmarks[ids["HIP"]].y]
    knee = [landmarks[ids["KNEE"]].x, landmarks[ids["KNEE"]].y]
    ankle = [landmarks[ids["ANKLE"]].x, landmarks[ids["ANKLE"]].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(hip, knee, ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        # is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        # is_bent_correctly = False

    # return feedback, is_bent_correctly
    return feedback, knee_angle

In [33]:
def classify_grade(score: float) -> str:
    if score >= 85: return "Perfect"
    if score >= 70: return "Good"
    if score >= 55: return "Fair"
    return "Poor"

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

In [34]:
def compute_rep_score_simple(trk: dict):
    if (trk["knee_angle_min"] is None or
        trk["hip_init"] is None or
        trk["hip_bottom"] is None or
        trk["knee_bottom"] is None):
        return 0.0, {}, "Invalid"
    knee_angle = trk["knee_angle_min"]
    back_dev = trk["back_dev_max"]
    hip_init = trk["hip_init"]
    depth_raw = trk["hip_bottom"] - hip_init
    knee_span = trk["knee_bottom"] - hip_init if trk["knee_bottom"] > hip_init else 0
    depth_ratio = depth_raw / knee_span if knee_span > 1e-6 else 0.0
    knee_norm = _clamp((KNEE_MAX_OK - knee_angle)/(KNEE_MAX_OK - KNEE_MIN_OK),0,1)
    back_norm = _clamp(1 - back_dev / BACK_DEV_TOL,0,1)
    depth_norm = _clamp(1 - abs(depth_ratio - IDEAL_DEPTH_RATIO)/IDEAL_DEPTH_RATIO,0,1)
    ctrl_norm = _clamp(trk["sit_frames"]/TARGET_SIT_FRAMES,0,1)
    score = 100*(W_KNEE*knee_norm + W_BACK*back_norm + W_DEPTH*depth_norm + W_CTRL*ctrl_norm)
    grade = classify_grade(score)
    breakdown = {
        "knee%": round(knee_norm*100,1),
        "back%": round(back_norm*100,1),
        "depth%": round(depth_norm*100,1),
        "ctrl%": round(ctrl_norm*100,1),
        "knee_angle": round(knee_angle,1),
        "depth_ratio": round(depth_ratio,2),
        "back_dev": round(back_dev,3),
        "sit_frames": trk["sit_frames"]
    }
    return round(score,1), breakdown, grade

In [35]:
def put_text_with_bg(img, text, org, color, font=cv2.FONT_HERSHEY_SIMPLEX,
                     scale=0.8, thickness=2, bg_color=(30,30,30), alpha=0.6, pad=6):
    (tw, th), bl = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    # 배경 영역
    bg_tl = (x - pad, y - th - pad)
    bg_br = (x + tw + pad, y + bl + pad//2)
    # 반투명 배경
    overlay = img.copy()
    cv2.rectangle(overlay, bg_tl, bg_br, bg_color, -1)
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
    # 텍스트
    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)

In [36]:
def draw_feedback(image, feedback, state, squat_count, extra_lines=None):
    put_text_with_bg(
        image,
        f"State: {state} | Count: {squat_count}",
        (30, 50),
        (0, 255, 0),  # 상태/카운트는 녹색 유지
        scale=0.9
    )
    if feedback:
        put_text_with_bg(
            image,
            feedback,
            (30, 95),
            FEEDBACK_COLOR,
            scale=0.85
        )
    if extra_lines:
        y = 140
        for line in extra_lines:
            put_text_with_bg(
                image,
                line,
                (30, y),
                EXTRA_COLOR,
                scale=0.65
            )
            y += 38

In [37]:
def resize_image(image, max_width):
    h, w = image.shape[:2]
    if w == max_width:
        return image
    scale = max_width / w
    return cv2.resize(image, (max_width, int(h * scale)), interpolation=cv2.INTER_LINEAR)

In [ ]:
def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
    global track, rep_scores, rep_grades, rep_breakdowns, avg_score, recent_rep_feedback
    ids = SIDE_IDX[side]
    hip_y = landmarks[ids["HIP"]].y
    knee_y = landmarks[ids["KNEE"]].y
    shoulder_y = landmarks[ids["SHOULDER"]].y

    # 초기 hip 기록 (1회)
    if initial_hip_y_dict[side] is None:
        initial_hip_y_dict[side] = hip_y
        print(f"Initial Hip Y: {hip_y:.3f}")
    init_hip = initial_hip_y_dict[side]

    visible_ok, vis_map = joints_visible(landmarks, side)
    if not visible_ok:
        feedback = "Low visibility"
        extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]
        return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

    # 속도 히스토리
    track["hip_history"].append(hip_y)
    if len(track["hip_history"]) > VEL_WINDOW:
        track["hip_history"].pop(0)
    if len(track["hip_history"]) >= 2:
        avg_vel = (track["hip_history"][-1]-track["hip_history"][0])/(len(track["hip_history"])-1)
        inst_vel = hip_y - track["hip_history"][-2]
    else:
        avg_vel = 0.0
        inst_vel = 0.0

    back_straightness = abs(hip_y - shoulder_y)
    feedback = ""
    extra = []

    # START
    if state == "START":
        feedback = "Check posture..."
        extra.append("Back: Good" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Back: Straighten!")
        if abs(hip_y - knee_y) < ENTER_SIT_GAP:
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y],
                "knee_angle_min": None,
                "back_dev_max": 0.0
            })
            print("START -> SIT")

    elif state == "SIT":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")

        # 최소 무릎 각도 / 허리 편차 갱신
        if track["knee_angle_min"] is None or knee_angle < track["knee_angle_min"]:
            track["knee_angle_min"] = knee_angle
        back_dev = abs(hip_y - shoulder_y)
        if back_dev > track["back_dev_max"]:
            track["back_dev_max"] = back_dev

        if hip_y > track["hip_bottom"]:
            track["hip_bottom"] = hip_y
            track["knee_bottom"] = knee_y

        track["sit_frames"] += 1
        depth = track["hip_bottom"] - init_hip
        if (not track["bottom_locked"] and
            track["sit_frames"] >= BOTTOM_MIN_FRAMES and
            depth >= DEPTH_MIN):
            track["bottom_locked"] = True
            extra.append("Bottom OK")

        if track["bottom_locked"] and inst_vel < UP_VEL_THRESH:
            state = "RISING"
            track["rise_frames"] = 0
            track["hip_history"] = [hip_y]
            track["rising_start_hip"] = track["hip_bottom"]
            print("SIT -> RISING")
        elif time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Timeout. Reset."
            print("SIT -> START (timeout)")

    elif state == "RISING":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")

        if track["knee_angle_min"] is None or knee_angle < track["knee_angle_min"]:
            track["knee_angle_min"] = knee_angle
        back_dev = abs(hip_y - shoulder_y)
        if back_dev > track["back_dev_max"]:
            track["back_dev_max"] = back_dev

        start_hip = track.get("rising_start_hip", track.get("hip_bottom", hip_y))
        target_y = min(init_hip + STAND_HIP_MARGIN, knee_y - KNEE_GAP)
        total_needed = start_hip - target_y
        if total_needed < 1e-6:
            total_needed = 1e-6
        rising_disp = start_hip - hip_y
        progress = rising_disp / total_needed
        progress = max(0.0, min(1.2, progress))
        extra.append(f"prog:{progress*100:4.0f}%")

        stand_ready = (
            hip_y < knee_y - KNEE_GAP and
            hip_y <= init_hip + STAND_HIP_MARGIN and
            (track["rise_frames"] >= UP_MIN_FRAMES or progress >= STAND_PROGRESS)
        )
        if stand_ready:
            # DEBUG: 스코어 입력 필드 확인
            print("[DEBUG SCORE INPUT]",
                  "knee_angle_min=", track.get("knee_angle_min"),
                  "hip_init=", track.get("hip_init"),
                  "hip_bottom=", track.get("hip_bottom"),
                  "knee_bottom=", track.get("knee_bottom"))

            score, breakdown, grade = compute_rep_score_simple(track)  # <-- 기존 스코어 계산 줄
            print(f"[REP] #{squat_count+1} score={score} grade={grade} breakdown={breakdown}")


            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            track["hip_history"] = [hip_y]

            # --- 반복 점수 계산 ---
            score, breakdown, grade = compute_rep_score_simple({
                "knee_angle_min": track["knee_angle_min"],
                "back_dev_max": track["back_dev_max"],
                "hip_init": track["hip_init"],
                "hip_bottom": track["hip_bottom"],
                "knee_bottom": track["knee_bottom"],
                "sit_frames": track["sit_frames"]
            })
            rep_scores.append(score)
            rep_grades.append(grade)
            rep_breakdowns.append(breakdown)
            global avg_score, recent_rep_feedback
            avg_score = round(sum(rep_scores)/len(rep_scores),1)
            recent_rep_feedback = f"Rep {squat_count}: {score} ({grade})"

            print(f"[REP] #{squat_count} score={score} grade={grade} breakdown={breakdown}")

            # 다음 rep 준비 변수 리셋
            track["knee_angle_min"] = None
            track["back_dev_max"] = 0.0
            track["sit_frames"] = 0
            track["bottom_locked"] = False
            track["rise_frames"] = 0
            track["hip_history"] = [hip_y]
            print("RISING -> STAND")


            # track["hip_history"] = [hip_y]
            # print("RISING -> STAND")
            return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

        if inst_vel < UP_VEL_THRESH:
            track["rise_frames"] += 1
            feedback = "Rising..."
        else:
            # 불완전 카운트 (원치 않으면 이 블록 제거)
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            extra.append("Incomplete")
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y],
                "knee_angle_min": None,
                "back_dev_max": 0.0
            })
            print("RISING -> SIT (incomplete)")
            return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra
        extra.append(f"riseF:{track['rise_frames']}")
        extra.append(f"vel:{avg_vel:.4f}")

    elif state == "STAND":
        feedback = "Standing: Good posture" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Standing: Straighten back"
        # if avg_vel > abs(UP_VEL_THRESH) and hip_y >= knee_y - ENTER_SIT_GAP:
        if inst_vel > 0 and hip_y >= knee_y - ENTER_SIT_GAP:
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y],
                "knee_angle_min": None,
                "back_dev_max": 0.0
            })
            feedback = "Descend"
            print("STAND -> SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [39]:
cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW if USE_DIRECTSHOW else 0)
# cap = cv2.VideoCapture(squat_videoPath)

# 전역 변수 초기화
squat_count = 0
initial_hip_y_dict = {"left": None, "right": None}
state = "START"
sit_start_time = None

active_side = None
side_stability_counter = 0

# FPS 측정용
prev_time = time.time()
frame_count = 0
fps = 0.0

# 세션 변수
session_running = False
session_start_time = None
last_display_frame = None

# 시작 관련
training_active = False
ready_frames = 0
countdown_start = None #  카운트다운 시작 시간

# squat goal
target_reached = False
stand_hold = 0

with mp_pose.Pose(
    model_complexity=1,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    if not session_running and last_display_frame is not None and state == "FINISHED":
      freeze = last_display_frame.copy()
      y0 = 60
      cv2.putText(freeze, f"Completed {TARGET_REPS} Reps!", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2, cv2.LINE_AA)
      y0 += 40
      if rep_scores:
            cv2.putText(freeze, f"Avg Score: {avg_score}", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,255), 2, cv2.LINE_AA)
            y0 += 30
            # 각 반복 점수/등급 라인별 출력
            for i, (s, g) in enumerate(zip(rep_scores, rep_grades), start=1):
                cv2.putText(freeze, f"Rep {i}: {s} ({g})", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255,255,255), 2, cv2.LINE_AA)
                y0 += 26
      cv2.putText(freeze, "s : restart, q : quit",
                  (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2, cv2.LINE_AA)
      cv2.imshow("MediaPipe Pose", resize_image(freeze, MAX_WIDTH))

      key = cv2.waitKey(1) & 0xFF
      if key in (27, ord('q')):
        break
      if key == ord('s'):
        squat_count = 0
        target_reached = False
        stand_hold = 0
        state = "START"
        session_running = True
        session_start_time = time.time()
        last_display_frame = None
        track.update({"hip_init":None,"hip_bottom":None,"knee_bottom":None,
                          "sit_frames":0,"bottom_locked":False,"rise_frames":0,
                          "hip_history":[], "knee_angle_min":None,"back_dev_max":0.0})
        # 이전 기록 지우고 새로 시작하려면 아래 해제
        rep_scores.clear(); rep_grades.clear(); rep_breakdowns.clear()
        # global avg_score, recent_rep_feedback
        stand_hold = 0
        print("[Session] RESTART")
      continue

    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break

    # BGR → RGB 변환
    frame.flags.writeable = False
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    feedback = ""  # 피드백 초기화
    extra_lines = []

    if session_running and results.pose_landmarks:
      landmarks = results.pose_landmarks.landmark
      active_side, side_stability_counter, left_sum, right_sum = check_side(
        landmarks, active_side, side_stability_counter
      )

      if active_side is not None:
        state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra_lines = process_state(landmarks, active_side, state, initial_hip_y_dict, sit_start_time, squat_count)

      if squat_count >= TARGET_REPS:
        if not target_reached:
          target_reached = True
          stand_hold = 0
        if state == "STAND":
          stand_hold += 1
        else:
          stand_hold = 0
        extra_lines.append(f"FinishHold: {stand_hold}/{TARGET_REP_HOLD_FRAMES}")
        feedback = f"Target Reps ({TARGET_REPS}) achieved!"
        if stand_hold >= TARGET_REP_HOLD_FRAMES:
          session_running = False
          state = "FINISHED"
          last_display_frame = frame.copy()
    elif not session_running:
      feedback = "stopped"

    frame.flags.writeable = True
    frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    # 피드백 및 상태 정보 표시
    draw_feedback(frame, feedback, state, squat_count, extra_lines)
    # if session_running and results.pose_landmarks and active_side and 'track' in globals():
    #     draw_stand_guides(frame, track, active_side, results.pose_landmarks.landmark)
    # if recent_rep_feedback:
    #   put_text_with_bg(frame, recent_rep_feedback, (30, frame.shape[0]-150), (255,255,255), scale=0.7)
    #   put_text_with_bg(frame, f"Avg Score: {avg_score}", (30, frame.shape[0]-115), (0,255,255), scale=0.65)

    # FPS 계산 (0.5초마다 갱신)
    frame_count += 1
    now = time.time()
    if now - prev_time >= 0.5:
      fps = frame_count / (now - prev_time)
      prev_time = now
      frame_count = 0
    cv2.putText(frame, f"FPS: {fps:.1f}", (30, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

    if not session_running and state != "FINISHED":
      cv2.putText(frame, "s : start, goal : {TARGET_REPS}", (30, frame.shape[0]-55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (30, 180, 255), 2, cv2.LINE_AA)
    else:
      cv2.putText(frame, "e : quit",
                  (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (30, 180, 255), 2, cv2.LINE_AA)

    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(frame, MAX_WIDTH)

    if session_running:
      last_display_frame = frame.copy()

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)

    key = cv2.waitKey(1) & 0xFF
    if key in (27, ord('q')):
      break
    if key == ord('s') and not session_running:
      session_running = True
      session_start_time = time.time()
    if key == ord('e') and session_running:
      session_running = False
      last_display_frame = frame.copy()

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 0.546
START -> SIT
SIT -> RISING
[DEBUG SCORE INPUT] knee_angle_min= 59.425742058035176 hip_init= None hip_bottom= 0.9010680913925171 knee_bottom= 0.9607109427452087
[REP] #1 score=0.0 grade=Invalid breakdown={}
[REP] #1 score=0.0 grade=Invalid breakdown={}
RISING -> STAND
STAND -> SIT
SIT -> RISING
[DEBUG SCORE INPUT] knee_angle_min= 55.8162757521743 hip_init= None hip_bottom= 0.9044018387794495 knee_bottom= 0.9518832564353943
[REP] #2 score=0.0 grade=Invalid breakdown={}
[REP] #2 score=0.0 grade=Invalid breakdown={}
RISING -> STAND
STAND -> SIT
SIT -> RISING
[DEBUG SCORE INPUT] knee_angle_min= 58.15441495400261 hip_init= None hip_bottom= 0.9014905691146851 knee_bottom= 0.9379953145980835
[REP] #3 score=0.0 grade=Invalid breakdown={}
[REP] #3 score=0.0 grade=Invalid breakdown={}
RISING -> STAND
